# Model Selection 모듈 소개
사이킷런의 `model_selection` 모듈은 다음과 같은 기능을 제공하는 함수 및 클래스를 제공한다.
- 학습 데이터와 테스트 데이터 세트를 분리
- 교차 검증 분할 및 평가
- Estimator의 하이퍼 파라미터를 튜닝 

## 1. 학습/테스트 데이터 세트 분리 - train_test_split()
학습과 예측을 위한 데이터 세트로 동일한 데이터 세트를 사용하면 제대로된 예측을 수행할 수 없다. 이렇게 되면 정확도가 100%로 마치 '시험지 정답을 보고 시험을 본 것'과 같다. 그러므로 데이터 세트를 학습용 데이터 세트와 예측을 위한 테스트 데이터 세트로 나누어야 한다.

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()    # 데이터셋 로딩
dt_clf = DecisionTreeClassifier()    # 의사결정트리 객체
train_data = iris.data       # feature 데이터 값
train_label = iris.target    # 결과 데이터 값
dt_clf.fit(train_data, train_label)    # 학습

# 학습 데이터 세트로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도 : ', accuracy_score(train_label, pred))

예측 정확도 :  1.0


데이터 세트를 학습용 데이터 세트와 테스트용 데이터 세트로 분리할 때 `train_test_split()`를 이용한다.

train_test_split(데이터.data, 데이터.target, test_size=0.3, random_state=131)

- `test_size` : 테스트 데이터 세트 크기 (디폴드값은 0.25)
- `train_size` : 학습용 데이터 세트 크기 (test_size로 비율을 설정하므로 거의 쓰이지 않음)
- `shuffle` : 데이터 분리 전에 데이터를 미리 섞을지 결정 (디폴드값은 True)
- `random_state` : 동일한 학습/테스트용 데이터 세트를 생성하기 위한 seed값 (`train_test_split()` 호출할 때 마다 seed값이 달라지므로 동일하게 분리하기 위한 같은 값으로 지정)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

# 테스트 데이터 셋은 30%
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=121)
dt_clf.fit(X_train, y_train)     # 학습
pred = dt_clf.predict(X_test)    # 테스트용 데이터셋으로 예측
print('예측 정확도 : {0:4f}'.format(accuracy_score(y_test, pred)))

예측 정확도 : 0.955556


학습을 위한 데이터의 양을 일정 수준 이상으로 보장하는 것도 좋지만, 학습된 모델을 가지고 예측 성능을 평가해보는 것도 중요하다.

## 2. 교차 검증
Overfitting이란 학습된 모델이 **학습된 데이터에만 과도하게 최적화**되어 실제 예측 시 새로운 데이터를 넣으면 성능이 떨어지는 것을 말한다. Overfitting은 똑같은 데이터 세트로 계속해서 반복 학습을 할 경우 발생하며 이는 여러 세트로 구성된 학습 데이터과 검증 데이터 세트를 통해 학습과 평가를 하는 **교차 검증**으로 해결할 수 있다.     
즉, 교차 검증은 테스트 데이터 세트와 학습 데이터 세트로 분리한 뒤, 학습 데이터 세트의 일부를 **검증 데이터 세트**로 두어 미리 평가를 하는 것이다. 최종 평가는 테스트 데이터 세트로 한다. 

### 2.1. K 폴드 교차 검증
가장 보편적인 교차 검증 기법으로 K개의 데이터 폴드 세트를 만들어 K번만큼 각 폴드 세트에 대해 학습과 검증 평가를 반복적으로 수행하는 방법이다.   
1. K개의 폴드된 데이터 세트를 학습 데이터 세트와 검증 데이터 세트로 나눈다.
2. 나누어진 데이터 세트로 K번 평가를 수행한다.
3. K개의 평가의 평균을 가지고 예측 성능을 평가한다.    

![K 폴드 교차 검증](./k_fold_cross_validation.png)

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 저장하는 리스트 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기 : ', features.shape[0])

붓꽃 데이터 세트 크기 :  150


`split()`을 호출하면 데이터 세트를 K개의 폴드 데이터 세트로 분리한다. 그 때 학습용/검증용 데이터로 분할한 인덱스들을 `ndarray` 형태로 반환한다.

In [5]:
n_iter = 0

# KFold 객체의 split()를 호출하면 폴드 별 학습용, 검증용 테스트의 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
    # kfold.split()에서 반환된 인덱스로 학습용, 검증용 데이터를 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    # 반복할 때마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'.format(n_iter, accuracy, train_size, test_index))
    print('#{0} 검증 세트 : {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration 별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도 : ', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#1 검증 세트 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
#2 검증 세트 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
#3 검증 세트 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
#4 검증 세트 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 1

### 2.2. Stratified K 폴드
![Stratified K Fold](./stratified_k_fold.png)

Stratified K 폴드는 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식이다. 불균형한 분포도를 가진 레이블 데이터 집합은 **특정 레이블 값이 특이하게 많거나 매우 적어서 분포가 한쪽으로 치우치는 것**을 말한다.  
예를 들어 스팸 메일일 경우 1을, 아닐 경우 0인 레이블 데이터 있다고 하자. 스팸 메일일 확률을 0.0001%로 데이터 세트의 극히 일부이다. 만약 이 0.0001%가 데이터 세트의 한 쪽에 몰려 있다면, 모델은 스팸 메일이 아닌 걸 주로 학습을 할 것이다.    

이를 방지하기 위해 **원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습과 검증 데이터 세트를 분배**하는 Stratified K 폴드 방식을 적용하면 된다.

In [6]:
import pandas as pd
import numpy as np

# 레이블 분포도 확인
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)    # feature 데이터 값을 DataFrame으로 생성
iris_df['label']=iris.target       # 레이블 데이터를 저장할 'lable' column 생성 및 값 저장 
iris_df['label'].value_counts()    # 각 레이블 값의 개수

2    50
1    50
0    50
Name: label, dtype: int64

In [7]:
# K Fold 방식 적용시 학습/검증 레이블 데이터 값의 분포도 확인
kfold = KFold(n_splits=3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('\n## 교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())


## 교차 검증 : 1
학습 레이블 데이터 분포 : 
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    50
Name: label, dtype: int64

## 교차 검증 : 2
학습 레이블 데이터 분포 : 
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    50
Name: label, dtype: int64

## 교차 검증 : 3
학습 레이블 데이터 분포 : 
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    50
Name: label, dtype: int64


위의 결과를 보았을 때 검증 데이터의 레이블 값은 학습 데이터의 레이블 값에 없는 값이므로 결국 검증 정확도가 0이 되어 버린다. 그래서 레이블 값의 분포도를 반영하여 검증을 하는 Stratified KFold 방법을 사용하면 이 문제를 해결할 수 있다.     
Stratified K Fold 방식은 KFold 방식은 딱 한 가지가 다른데 레이블 데이터 분포도에 따라 학습/검증 데이터를 나누기 때문에 **`split()` 메서드의 인자로 피처 데이터 세트뿐만 아니라 레이블 데이터 세트도 포함**해야 한다.

In [8]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포 : 
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포 : 
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포 : 
 2    34
1    34
0    34
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    16
1    16
0    16
Name: label, dtype: int64


In [9]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy = []

# 레이블 값 분포도를 파악해야 하므로 label 데이터도 넘겨주어야 한다
for train_index, test_index in skfold.split(features, label):
    # split()으로 반환된 인덱스를 이용해 학습/검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복할 때마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도 : ', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도 : ', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 0.9804, 학습 데이터 크기 : 99, 검증 데이터 크기 : 51
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]

#2 교차 검증 정확도 : 0.9216, 학습 데이터 크기 : 99, 검증 데이터 크기 : 51
#2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133]

#3 교차 검증 정확도 : 0.9792, 학습 데이터 크기 : 102, 검증 데이터 크기 : 48
#3 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 :  [0.9804 0.9216 0.9792]
## 평균 검증 정확도 :  0.9604


* 불균등한 분포의 레이블 데이터 세트를 가진다면 반드시 **Stratified K 폴드**를 이용해 교차검증을 해야 한다. 
* 분류(Classification)에서는 **Stratified K 폴드**로 분할되어야 한다. 
* 회귀(Regression)의 경우 결과값이 연속된 숫자값이기 때문에 결과값의 분포도를 고려할 필요가 없으므로 Stratified K 폴드가 지원되지 않는다.

### 2.3. 교차 검증을 보다 편하게 - `cross_val_score()`

사이킷런은 교차 검증을 좀 더 편리하게 수행할 수 있는 API를 제공하는데 `cross_val_score()`를 사용하면 보다 쉽게 교차 검증을 할 수 있다. `cross_val_score()`는 아래 3가지 과정을 한 번에 수행해주는 함수이다.
1. 폴드 세트를 설정
2. for문에서 반복적으로 학습 및 테스트 데이터의 인덱스를 추출 
3. 반복적으로 학습과 예측을 수행
4. 예측 성능 반환

**cross_val_score(estimator, X, y=None, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs')**
- `estimator` : Classifier나 Regressor
- `X` : 피처 데이터 세트
- `y` : 레이블 데이터 세트
- `scoring` : 예측 성능 평가 지표
- `cv` : 교차 검증 폴드 수    

`scoring`으로 지정된 성능 지표 측정값을 배열 형태로 반환한다. 이 때 `classifier`가 입력되면 Stratified K 폴드 방식으로 분할을 하고 `regressor`가 입력되면 Stratified K 폴드 방식을 쓸 수 없으므로 K 폴드 방식으로 분할한다.

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 accuracy, 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도 : ', np.round(scores, 4))
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.9804 0.9216 0.9792]
평균 검증 정확도 :  0.9604


`cross_val_score()`는 하나의 평가 지표만 가능하다. 만약 여러 개의 평가 지표를 반환하고 싶을 경우 `cross_validate()`를 사용하면 되며 수행시간도 같이 제공된다.

## 3. GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에
하이퍼 파라미터란 머신러닝 알고리즘을 구성하는 주요 구성 요소이며, 이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있다. `GridSearchCV`를 이용하면 `Classifier`나 `Regressor`와 같은 알고리즘에 하이퍼 파라미터를 **순차적으로** 입력하면서 최적의 파라미터를 도출해낸다.    
* 사용자가 튜닝하고자 하는 어려 개의 하이퍼 파라미터를 다양하게 테스트하면서 쉽게 최적의 파라미터를 찾을 수 있다.
* 하지만 순차적으로 테스트하므로 수행시간이 오래 걸린다.
* `CV=3`, `max_depth=[1,2,3]`, `min_samples_split=[2,3]`이면 `3*3*2=18`의 학습/평가가 이루어진다.

**GridSearchCV(estimator=`머신러닝 알고리즘`, param_grid=`테스트할 하이퍼 파라미터`, scoring=`예측 성능 지표`, cv=`폴드수`, refit=`True/False`)**
* `estimator` : classifer, regressor, pipeline
* `param_grid` : 문자열 key값과 리스트 value값을 가진 딕셔너리 형태이며, 튜닝을 위한 파라미터 명과 값들을 지정
* `scoring` : 예측 성능을 측정할 평가 방법
* `cv` : 교차 검증을 위해 분할되는 학습/테스트 세트의 개수
* `refit` : `True`이면 최적의 하이퍼 파라미터로 `estimator` 객체를 재학습시킴 (디폴트값은 `True`)

In [11]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

iris_data = load_iris()    # 데이터 로딩

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

학습 데이터 세트인 `X_train`을 GridSearchCV객체의 `fit`의 인자로 입력하면 다음과 같은 과정이 일어난다.
1. `cv`에 기술된 폴딩 세트로 학습데이터를 분할
2. `param_grid`에 기술된 하이퍼 파라미터를 순차적으로 변경하면서 학습/평가 수행
3. `cv_results_` 속성에 결과를 기록

In [13]:
import pandas as pd

# param_grid의 하이퍼 파라미터를 3개의 학습/테스트 폴드 세트로 나누어 설정
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
score_df = pd.DataFrame(grid_dtree.cv_results_)
score_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


* `param` : 수행할 때 마다 적용된 개별 하이퍼 파라미터값
* `rank_test_score` : 하이퍼 파라미터별로 성능이 좋은 score 순위
* `mean_test_score` : 개별 파라미터별로 각 폴드 데이터 세트의 평가 결과의 평균

In [14]:
print('GridSearchCV 최적 파라미터 : ', grid_dtree.best_params_)              # 최고 성능을 나타낸 하이퍼 파라미터 값
print('GridSearchCV 최고 적확도 : {0:.4f}'.format(grid_dtree.best_score_))   # 그 때의 평과 결과값

GridSearchCV 최적 파라미터 :  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 적확도 : 0.9750


In [15]:
# GridSearchCV의 refit으로 이미 학습된 estimator 변환
estimator = grid_dtree.best_estimator_     # 최적의 하이퍼 파라미터로 학습한 estimator

# GridSearchCV의 best_estimator_는 이미 최적 학습이 되었으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도 : {0:4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도 : 0.966667


일반적으로 `GridSearchCV`를 이용해 최적 하이퍼 파라미터 튜닝을 한 뒤에 별도의 테스트 데이터 세트에서 이를 평가하는 것이 일반적이다.